<a href="https://colab.research.google.com/github/sarim711/image_segmentation_ISIC2016/blob/main/Image_Segmentation_ISIC2016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Source: https://github.com/zekikus/MedSegBench

# Install medsegbench lib
!pip install medsegbench

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.0 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=0c5a5a1e909af47d9dd496e6d6b30d52a8ea2759457be0502b3d72f139ea952d
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=938cdc345d0f21678f387c652b99032dabd63bbcd61ff6c4f35f486515a4a154
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch

In [ ]:
# import lib and utils from related to the ISIC 2016 dataset (melanoma segmentation)
import medsegbench
from medsegbench import Isic2016MSBench
from medsegbench import INFO

In [ ]:
# Get info from the dataset
info = INFO["isic2016"]
n_channels = info['n_channels_im']
n_classes = len(info['pixel_labels'])
n_samples = info['n_samples']

DataClass = getattr(medsegbench, info['python_class'])

In [ ]:
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

In [ ]:
# dataset loading using the original DataClass
BATCH_SIZE = 32

img_size = 128
download = True


In [ ]:
class AugmentedDataClass(DataClass):
    def __init__(self, split, size=128, transform=None, target_transform=None, download=False):
        super().__init__(split=split, size=size, transform=transform, target_transform=target_transform, download=download)
        self.split = split

        # Define augmentations
        self.image_augmentations = transforms.Compose([
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Only for image, as segmentation  masks are arrays of class label or binary masks)
            transforms.RandomRotation(degrees=30),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip()
        ])

        self.mask_augmentations = transforms.Compose([  # Geometric transformations for masks same as image
            transforms.RandomRotation(degrees=30),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip()
        ])

    def __getitem__(self, idx):
        image, mask = super().__getitem__(idx)

        if self.split == 'train':
            seed = torch.randint(0, 2**32, (1,)).item()  # Generate a random seed
            torch.manual_seed(seed)

            # Apply same geometric transformations to both image and mask
            image = self.image_augmentations(image)  # Includes color jitter
            torch.manual_seed(seed)
            mask = self.mask_augmentations(mask)  # Excludes color jitter

        return image, mask


# Define transformations
data_transform = transforms.Compose([transforms.ToTensor()])

# Load datasets
train_dataset = AugmentedDataClass(split='train', size=img_size, transform=data_transform, target_transform=data_transform, download=download)
validation_dataset = AugmentedDataClass(split='val', size=img_size, transform=data_transform, target_transform=data_transform, download=download)
test_dataset = AugmentedDataClass(split='test', size=img_size, transform=data_transform, target_transform=data_transform, download=download)

# Dataloader
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = data.DataLoader(dataset=validation_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)


Using downloaded and verified file: /root/.medsegbench/isic2016_128.npz
Using downloaded and verified file: /root/.medsegbench/isic2016_128.npz
Using downloaded and verified file: /root/.medsegbench/isic2016_128.npz


In [ ]:
# model config
NUM_EPOCHS = 10
lr = 0.001

In [ ]:
# Unet Model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        def conv_block(input_channels, output_channels):
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True),
                nn.Conv2d(output_channels, output_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True)
            )

        self.encoder1 = conv_block(in_channels, 64)
        self.encoder2 = conv_block(64, 128)
        self.encoder3 = conv_block(128, 256)
        self.encoder4 = conv_block(256, 512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = conv_block(512, 1024)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = conv_block(1024, 512)

        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = conv_block(512, 256)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = conv_block(256, 128)

        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = conv_block(128, 64)

        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoding path
        encoder1 = self.encoder1(x)
        encoder2 = self.encoder2(self.pool(encoder1))
        encoder3 = self.encoder3(self.pool(encoder2))
        encoder4 = self.encoder4(self.pool(encoder3))

        # Bottleneck
        bottleneck = self.bottleneck(self.pool(encoder4))

        # Decoding path
        decoder4 = self.upconv4(bottleneck)
        decoder4 = torch.cat((decoder4, encoder4), dim=1)
        decoder4 = self.decoder4(decoder4)

        decoder3 = self.upconv3(decoder4)
        decoder3 = torch.cat((decoder3, encoder3), dim=1)
        decoder3 = self.decoder3(decoder3)

        decoder2 = self.upconv2(decoder3)
        decoder2 = torch.cat((decoder2, encoder2), dim=1)
        decoder2 = self.decoder2(decoder2)

        decoder1 = self.upconv1(decoder2)
        decoder1 = torch.cat((decoder1, encoder1), dim=1)
        decoder1 = self.decoder1(decoder1)

        return self.out_conv(decoder1)

# Test the model
model = UNet(in_channels=3, out_channels=1)
x = torch.randn(1, 3, 256, 256)
output = model(x)
print(f"Model output shape: {output.shape}")


Model output shape: torch.Size([1, 1, 256, 256])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

criterion = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
def calculate_pixel_ratios(dataset):
    foreground_pixels = 0
    background_pixels = 0

    for _, mask in dataset:
        mask = mask.squeeze()  # Remove batch dimension if present
        foreground_pixels += (mask > 0).sum().item()  # Count positive (foreground) pixels
        background_pixels += (mask == 0).sum().item()  # Count zero (background) pixels

    total_pixels = foreground_pixels + background_pixels
    foreground_ratio = foreground_pixels / total_pixels
    background_ratio = background_pixels / total_pixels

    return foreground_ratio, background_ratio

# Example usage
foreground_ratio, background_ratio = calculate_pixel_ratios(train_dataset)
print(f"Foreground Ratio: {foreground_ratio:.4f}, Background Ratio: {background_ratio:.4f}")


Foreground Ratio: 0.2703, Background Ratio: 0.7297


In [ ]:

pos_weight_value = background_ratio / foreground_ratio
pos_weight = torch.tensor([pos_weight_value],device=device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)


In [ ]:
def dice_coefficient(preds, targets, threshold=0.5):
    # Apply threshold to convert logits to binary predictions
    preds = (torch.sigmoid(preds) > threshold).float()
    intersection = (preds * targets).sum()  # True positives
    union = preds.sum() + targets.sum()  # Total pixels in both masks
    dice = (2.0 * intersection) / (union + 1e-8)  # Add epsilon to avoid division by zero
    return dice.item()

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)

    for epoch in range(num_epochs):
        train_loss = []
        train_dice_scores = []

        model.train()

        # Training loop
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, targets)
            train_loss.append(loss.item())

            # Backward pass + optimization step
            loss.backward()
            optimizer.step()

            # Calculate Dice coefficient for training
            dice = dice_coefficient(outputs, targets)
            train_dice_scores.append(dice)

        # Compute average training metrics
        avg_train_loss = sum(train_loss) / len(train_loss)
        avg_train_dice = sum(train_dice_scores) / len(train_dice_scores)

        # Validation loop
        val_loss = []
        val_dice_scores = []

        model.eval()
        with torch.no_grad():  # Disable gradient computation for validation
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                # Forward pass
                outputs = model(inputs)

                # Compute loss
                loss = criterion(outputs, targets)
                val_loss.append(loss.item())

                # Calculate Dice coefficient for validation
                dice = dice_coefficient(outputs, targets)
                val_dice_scores.append(dice)

        # Compute average validation metrics
        avg_val_loss = sum(val_loss) / len(val_loss)
        avg_val_dice = sum(val_dice_scores) / len(val_dice_scores)

        # Print metrics for the epoch
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train Dice: {avg_train_dice:.4f}")
        print(f"  Val Loss: {avg_val_loss:.4f}, Val Dice: {avg_val_dice:.4f}")

train_model(model=model, train_loader=train_loader, val_loader=val_loader, criterion=criterion, optimizer=optimizer, num_epochs=NUM_EPOCHS, device=device)

Epoch 1/10
  Train Loss: 0.4438, Train Dice: 0.7712
  Val Loss: 0.4138, Val Dice: 0.7598
Epoch 2/10
  Train Loss: 0.4458, Train Dice: 0.7696
  Val Loss: 0.4159, Val Dice: 0.7576
Epoch 3/10
  Train Loss: 0.4457, Train Dice: 0.7691
  Val Loss: 0.4156, Val Dice: 0.7624
Epoch 4/10
  Train Loss: 0.4407, Train Dice: 0.7677
  Val Loss: 0.4181, Val Dice: 0.7453
Epoch 5/10
  Train Loss: 0.4414, Train Dice: 0.7696
  Val Loss: 0.4177, Val Dice: 0.7557
Epoch 6/10
  Train Loss: 0.4554, Train Dice: 0.7663
  Val Loss: 0.4094, Val Dice: 0.7598
Epoch 7/10
  Train Loss: 0.4413, Train Dice: 0.7713
  Val Loss: 0.4138, Val Dice: 0.7626
Epoch 8/10
  Train Loss: 0.4332, Train Dice: 0.7721
  Val Loss: 0.4180, Val Dice: 0.7591
Epoch 9/10
  Train Loss: 0.4368, Train Dice: 0.7737
  Val Loss: 0.4147, Val Dice: 0.7642
Epoch 10/10
  Train Loss: 0.4319, Train Dice: 0.7737
  Val Loss: 0.4200, Val Dice: 0.7588


In [ ]:
from torchvision.models import resnet34, ResNet34_Weights
import torch.nn.functional as F # import functional interface
class UNetWithResNetEncoder(nn.Module):
    def __init__(self, out_channels=1):
        super(UNetWithResNetEncoder, self).__init__()

        # Pre-trained ResNet backbone
        resnet = resnet34(weights=ResNet34_Weights.DEFAULT)
        self.encoder1 = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu)  # First layer
        self.encoder2 = resnet.layer1  # ResNet block 1
        self.encoder3 = resnet.layer2  # ResNet block 2
        self.encoder4 = resnet.layer3  # ResNet block 3

        # Modify the last ResNet block (layer4) to prevent excessive downsampling
        self.encoder5 = resnet.layer4
        for name, layer in self.encoder5.named_modules():
            if isinstance(layer, nn.Conv2d) and layer.stride == (2, 2):
                layer.stride = (1, 1)  # Reduce downsampling

        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = self.conv_block(1024, 512)

        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = self.conv_block(512, 256)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = self.conv_block(256, 128)

        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = self.conv_block(128, 64)

        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        # Encoding path
        encoder1 = self.encoder1(x)
        encoder2 = self.encoder2(F.max_pool2d(encoder1, 2))
        encoder3 = self.encoder3(F.max_pool2d(encoder2, 2))
        encoder4 = self.encoder4(F.max_pool2d(encoder3, 2))
        encoder5 = self.encoder5(F.max_pool2d(encoder4, 2))

        # Bottleneck
        bottleneck = self.bottleneck(encoder5)

        # Decoding path
        decoder4 = self.upconv4(bottleneck)
        decoder4 = torch.cat((decoder4, F.interpolate(encoder5, size=decoder4.shape[2:], mode="bilinear", align_corners=False)), dim=1)
        decoder4 = self.decoder4(decoder4)

        decoder3 = self.upconv3(decoder4)
        decoder3 = torch.cat((decoder3, F.interpolate(encoder4, size=decoder3.shape[2:], mode="bilinear", align_corners=False)), dim=1)
        decoder3 = self.decoder3(decoder3)

        decoder2 = self.upconv2(decoder3)
        decoder2 = torch.cat((decoder2, F.interpolate(encoder3, size=decoder2.shape[2:], mode="bilinear", align_corners=False)), dim=1)
        decoder2 = self.decoder2(decoder2)

        decoder1 = self.upconv1(decoder2)
        decoder1 = torch.cat((decoder1, F.interpolate(encoder2, size=decoder1.shape[2:], mode="bilinear", align_corners=False)), dim=1)
        decoder1 = self.decoder1(decoder1)

        # Final upsampling to match input size
        final_output = F.interpolate(decoder1, size=x.shape[2:], mode="bilinear", align_corners=False)

        return self.out_conv(final_output)


# Instantiate and test the updated model
model_resnet_unet = UNetWithResNetEncoder(out_channels=1)
x = torch.randn(1, 3, 256, 256)
output = model_resnet_unet(x)
print(f"Output shape: {output.shape}")


Output shape: torch.Size([1, 1, 256, 256])


In [ ]:
# Train the model with training and validation
NUM_EPOCHS = 10
lr = 0.001

# Initialize model, criterion, optimizer, and dataloaders
model_resnet_unet = UNetWithResNetEncoder(out_channels=1).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model_resnet_unet.parameters(), lr=lr)

train_model(model=model_resnet_unet, train_loader=train_loader, val_loader=val_loader, criterion=criterion, optimizer=optimizer, num_epochs=NUM_EPOCHS, device=device)

Epoch 1/10
  Train Loss: 0.5709, Train Dice: 0.7154
  Val Loss: 0.5911, Val Dice: 0.6221
Epoch 2/10
  Train Loss: 0.4539, Train Dice: 0.7731
  Val Loss: 0.3926, Val Dice: 0.7763
Epoch 3/10
  Train Loss: 0.4306, Train Dice: 0.7715
  Val Loss: 0.3492, Val Dice: 0.7777
Epoch 4/10
  Train Loss: 0.3933, Train Dice: 0.7935
  Val Loss: 0.3183, Val Dice: 0.8072
Epoch 5/10
  Train Loss: 0.3994, Train Dice: 0.7966
  Val Loss: 0.3340, Val Dice: 0.8152
Epoch 6/10
  Train Loss: 0.3917, Train Dice: 0.7916
  Val Loss: 0.4314, Val Dice: 0.7198
Epoch 7/10
  Train Loss: 0.3826, Train Dice: 0.7991
  Val Loss: 0.3219, Val Dice: 0.8401
Epoch 8/10
  Train Loss: 0.3770, Train Dice: 0.7954
  Val Loss: 0.3620, Val Dice: 0.7675
Epoch 9/10
  Train Loss: 0.3799, Train Dice: 0.8010
  Val Loss: 0.3484, Val Dice: 0.7769
Epoch 10/10
  Train Loss: 0.3594, Train Dice: 0.8077
  Val Loss: 0.3835, Val Dice: 0.7544


In [ ]:
def predict(model, image, device, threshold=0.5):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        image = image.to(device)  # Move the image to the specified device
        output = model(image)  # Forward pass through the model
        output = torch.sigmoid(output)  # Apply sigmoid to get probabilities
        mask = (output > threshold).float()  # Binarize using the threshold
    return mask.squeeze(0).squeeze(0)  # Remove batch and channel dimensions

def evaluate_on_test(model, test_loader, device):
    model.to(device)
    model.eval()
    dice_scores = []

    with torch.no_grad():
        for images, masks in test_loader:
            images, masks = images.to(device), masks.to(device)  # Move data to the device

            # Make predictions
            outputs = model(images)
            outputs = torch.sigmoid(outputs)  # Convert logits to probabilities
            preds = (outputs > 0.5).float()  # Binarize predictions

            # Compute Dice coefficient
            dice = dice_coefficient(preds, masks)
            dice_scores.append(dice)

    avg_dice = sum(dice_scores) / len(dice_scores)
    print(f"Average Dice Coefficient on Test Dataset: {avg_dice:.4f}")

evaluate_on_test(model_resnet_unet, test_loader, device)

Average Dice Coefficient on Test Dataset: 0.7797
